## Stress-testing orbital with GBMs

### Goals

GBMs are a more complicated tree-based ensemble model than RandomForests insomuchas their predictions are not simply the "majority vote" of their leaf nodes. We did, in fact, go to all the trouble of boosting those gradients...

Instead, GBM predictions are calculated as `e^x / (1 + e^x)` or more simply `1 / (1 + e^-x)` where x is the result of summing the outputs of the trees (subject to the correct dampening parameters). This makes them a bit trickier for `orbital` to get right. 

Currently, this transformation is done correctly for `output_probabiliy.1` but incorrectly for the other variables `orbital` offers. Fortunately, the correct one is the one that practicioners will want 90% of the time, but there could be other cases like modeling propensity scores for an observational study where both positive and negative prediction fields are used. 

I opened an issue about this in the `orbital` repo. You can track it [here](https://github.com/posit-dev/orbital/issues/53)

### Set Up

In [ ]:
import orbital
import duckdb
import sqlglot
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

In [ ]:
#| label: setup

# mock data
X_train, y_train = make_classification(random_state=504)
X_train = X_train.round(3)

# mock pipeline
pipeline = Pipeline([
    ("prep", ColumnTransformer([("scaler", StandardScaler(), [])], remainder="passthrough")),
    ("gbm", GradientBoostingClassifier(max_depth = 1, n_estimators = 1, random_state=102)),
])
pipeline.fit(X_train, y_train)

# render SQL from orbital
n_cols = len(X_train[0])
nm_cols = [f"var_{i}" for i in range(n_cols)]
feat_dict = {}
for n in nm_cols:
    feat_dict[n] = orbital.types.DoubleColumnType()
orbital_pipeline = orbital.parse_pipeline(pipeline, features=feat_dict)
sql_raw = orbital.export_sql("DATA_TABLE", orbital_pipeline, dialect="duckdb")

c:\Users\emily\Desktop\orbital-exploration\.venv\Lib\site-packages\orbital\translation\steps\trees\classifier.py:135: FutureWarning: `case` is deprecated as of v10.0.0, removed in v11.0; use ibis.cases()
  ibis.case().when(condition, t_val).else_(f_val).end()
c:\Users\emily\Desktop\orbital-exploration\.venv\Lib\site-packages\orbital\translation\steps\trees\classifier.py:157: FutureWarning: `case` is deprecated as of v10.0.0, removed in v11.0; use ibis.cases()
  ibis.case()


### Isolating the incorrect outcome

Facially, we can see something isn't right because the output probabilities don't sum to 1. 

In [ ]:
#| label: valid-struct
DATA_TABLE = pd.DataFrame(X_train, columns = nm_cols)

# structure of table
df_preds = duckdb.sql(sql_raw).df()
print(df_preds.head())

   output_label  output_probability.0  output_probability.1
0             0              0.693682              0.545526
1             0              0.693682              0.545526
2             0              0.693682              0.545526
3             0              0.693682              0.545526
4             0              0.693682              0.545526
   output_label  output_probability.0  output_probability.1
0             0              0.693682              0.545526
1             0              0.693682              0.545526
2             0              0.693682              0.545526
3             0              0.693682              0.545526
4             0              0.693682              0.545526


Which value (or both) is wrong? The positive prediction case seems to be correct.

In [ ]:
#| label: valid-values
preds_orb = df_preds['output_probability.1']
preds_ppl     = pipeline.predict_proba(X_train)[:,1]
print(f"ppl and orb match: {np.all(np.isclose(preds_ppl, preds_orb))}")

ppl and orb match: True
ppl and orb match: True


### Diagnosis

We can get some insight by looking at the underlying query.

In [ ]:
sql_fmt = sqlglot.transpile(sql_raw, write="duckdb", identify=True, pretty=True)[0]
print(sql_fmt)

SELECT
  CAST(CASE
    WHEN CASE
      WHEN "t0"."var_6" <= 0.3644999861717224
      THEN -0.15555556118488312
      ELSE 0.1826086938381195
    END > 0.5
    THEN 1
    ELSE 0
  END AS BIGINT) AS "output_label",
  1 / (
    EXP(
      -(
        1.0 - CASE
          WHEN "t0"."var_6" <= 0.3644999861717224
          THEN -0.15555556118488312
          ELSE 0.1826086938381195
        END
      )
    ) + 1
  ) AS "output_probability.0",
  1 / (
    EXP(
      -CASE
        WHEN "t0"."var_6" <= 0.3644999861717224
        THEN -0.15555556118488312
        ELSE 0.1826086938381195
      END
    ) + 1
  ) AS "output_probability.1"
FROM "DATA_TABLE" AS "t0"
SELECT
  CAST(CASE
    WHEN CASE
      WHEN "t0"."var_6" <= 0.3644999861717224
      THEN -0.15555556118488312
      ELSE 0.1826086938381195
    END > 0.5
    THEN 1
    ELSE 0
  END AS BIGINT) AS "output_label",
  1 / (
    EXP(
      -(
        1.0 - CASE
          WHEN "t0"."var_6" <= 0.3644999861717224
          THEN -0.1555555611848831

Now, we can see where the logic has broken down. The SQL translation is:

- `output_label`: if {tree} > 0.5 then 1 else 0
- `output_probability.0`: 1 / (exp(1 - {tree}) + 1)
- `output_probability.1`: 1 / (exp(-tree) + 1)

Only the last of these correctly post-processes our tree:

- `output_probability.1` here is right. This is taking the function and applying the inverse logistic transformation. 
- `output_probability.0` here is wrong. What we want is `1 - transform(tree)` but instead we get `transform(1 - tree)`. Specifically, I believe the problem is that [this happens](https://github.com/posit-dev/orbital/blob/08302c9b1c403d209d00d7c5974b2fd17b51919b/src/orbital/translation/steps/trees/classifier.py#L166) before [this happens](https://github.com/posit-dev/orbital/blob/08302c9b1c403d209d00d7c5974b2fd17b51919b/src/orbital/translation/steps/linearclass.py#L99).
- `output_label` hese is also wrong. It ignores the transformation completely. 

Put another way, We can see that for output_probability.1 it is correctly applying the sigmoid transformation ( 1/( e^-z + 1) which is equivalent to e^z  / (e^z + 1)) . However, for output_probability.0 it is doing 1 / (e^-(1-z) + 1) instead of something equivalent to 1 / (e^z + 1) which would give the desired result.

Needless to say, if I swap in a RandomForestClassifier, this works as expected since no post-transformations are required.